In [ ]:
import os
import cv2
import torch
import random
import numpy as np
from torch.utils.data import Dataset

class EarDatasetStructured(Dataset):
    def __init__(self, root_dir="/Images", img_size=128, samples_per_folder=10):
        """
        root_dir: 'ear model/Images'
        Folder names look like: '001.ALI_HD', '002.LeDuong_BL', etc.
        """

        self.data = []
        self.img_size = img_size
        self.samples_per_folder = samples_per_folder

        # list folders sorted: 001..., 002..., ..., 164...
        folders = sorted(os.listdir(root_dir))

        for folder in folders:
            folder_path = os.path.join(root_dir, folder)
            if not os.path.isdir(folder_path):
                continue

            # extract numeric ID from folder name
            # example: "001.ALI_HD" → 1
            folder_num_str = folder.split(".")[0]  # '001'
            folder_num = int(folder_num_str)

            # label assignment
            if 1 <= folder_num <= 98:
                label = 0  # male
            else:
                label = 1  # female

            # collect all image paths
            img_files = [
                os.path.join(folder_path, f)
                for f in os.listdir(folder_path)
                if f.lower().endswith((".jpg", ".jpeg", ".png"))
            ]

            # sample a fixed number
            if len(img_files) > samples_per_folder:
                img_files = random.sample(img_files, samples_per_folder)

            for img_path in img_files:
                self.data.append((img_path, label))

        print(f"Loaded {len(self.data)} images from {len(folders)} folders")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]

        # Load image with OpenCV
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = img.astype(np.float32) / 255.0

        # HWC → CHW
        img = np.transpose(img, (2, 0, 1))

        img = torch.tensor(img)
        label = torch.tensor(label).long()

        return img, label


In [ ]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64×64

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32×32

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16×16
        )

        self.fc = nn.Sequential(
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


In [ ]:
import torch.optim as optim
from tqdm import tqdm

def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_acc = 0

    for imgs, labels in tqdm(loader):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        total_acc += (preds == labels).float().mean().item()

    return total_loss / len(loader), total_acc / len(loader)


In [ ]:
def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    total_acc = 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            total_acc += (preds == labels).float().mean().item()

    return total_loss / len(loader), total_acc / len(loader)


In [ ]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load dataset

dataset = EarDatasetStructured(
    root_dir="Images",
    img_size=128,
    samples_per_folder=20
)

# Train/Val split
idx_train, idx_val = train_test_split(
    np.arange(len(dataset)), test_size=0.2, shuffle=True, random_state=42
)

train_subset = torch.utils.data.Subset(dataset, idx_train)
val_subset = torch.utils.data.Subset(dataset, idx_val)

train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=16, shuffle=False)

# Model
model = SimpleCNN(num_classes=2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Train
for epoch in range(14):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f}")


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

def show_one_image_per_folder(root_dir="Images"):
    folders = sorted(os.listdir(root_dir))

    plt.figure(figsize=(12, 20))

    i = 1
    for folder in folders:
        folder_path = os.path.join(root_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        # find first image
        imgs = [f for f in os.listdir(folder_path) if f.lower().endswith((".jpg",".png",".jpeg"))]
        if len(imgs) == 0:
            continue

        img_path = os.path.join(folder_path, imgs[0])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.subplot(len(folders)//5 + 1, 5, i)
        plt.imshow(img)
        plt.title(folder)
        plt.axis("off")

        i += 1
        if i > 50:  # show first 50 for convenience
            break

    plt.tight_layout()
    plt.show()

# Call it:
show_one_image_per_folder("Images")


In [ ]:

def predict_gender(model, image_path, img_size=128):
    # Load with OpenCV
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Preprocess (same as training)
    img_resized = cv2.resize(img_rgb, (img_size, img_size))
    img_norm = img_resized.astype(np.float32) / 255.0
    img_chw = np.transpose(img_norm, (2, 0, 1))  # HWC → CHW

    # Create torch tensor
    input_tensor = torch.tensor(img_chw).unsqueeze(0).to(device)  # (1,3,H,W)

    # Predict
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output, dim=1).item()

    # Convert label to text
    label = "Male" if pred == 0 else "Female"

    # Show image + label
    cv2.putText(img_rgb, label, (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)

    plt.imshow(img_rgb)
    plt.title(f"Prediction: {label}")
    plt.axis("off")
    plt.show()

    return label


In [ ]:
torch.save(model.state_dict(), "ear_model.pth")


In [ ]:
import os

for i in range(8):
    test_image = f"img_{i}.png"
    if not os.path.exists(test_image):
        print(f"File not found: {test_image}")
    else:
        result = predict_gender(model, test_image)
        print(f"{test_image} -> {result}")


In [ ]:


test_image = "img.png"
predict_gender(model, test_image)

test_image2 = "img_1.png"
predict_gender(model, test_image2)


test_image3 = "img_2.png"
predict_gender(model, test_image3)


def test_one_from_each_folder(model, root_dir="Images"):
    folders = sorted(os.listdir(root_dir))

    for folder in folders:
        folder_path = os.path.join(root_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        imgs = [f for f in os.listdir(folder_path) if f.lower().endswith((".jpg",".png",".jpeg"))]
        if len(imgs) == 0:
            continue

        img_path = os.path.join(folder_path, imgs[0])

        print(f"{folder} → {predict_gender(model, img_path)}")

# Call it:
test_one_from_each_folder(model, "Images")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming these lists are filled during training:
# train_losses, val_losses, train_accs, val_accs
# And validation outputs: all_labels, all_preds

# --------- 1. Plot Loss & Accuracy ---------
plt.figure(figsize=(12,5))

# Loss
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()

# Accuracy
plt.subplot(1,2,2)
plt.plot(train_accs, label="Train Accuracy")
plt.plot(val_accs, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()

plt.tight_layout()
plt.show()

# --------- 2. Confusion Matrix ---------
cm = confusion_matrix(all_labels, all_preds)
acc = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {acc:.4f}")

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Male","Female"], yticklabels=["Male","Female"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
